# Working code

In [8]:
import ee
import pandas as pd
from datetime import datetime, timedelta, timezone
import pytz

# Initialize the Earth Engine module with the project ID
ee.Initialize(project='data690-zhouhaomatt')

# Define the points of interest
points_of_interest = [
    (170.89142642028511, -43.99924808088317),
    (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317),
    (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317),
    (170.89142642028511, -43.99915824976567),
    (170.89192593223774, -43.99915824976567),
    (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]

# Convert points to EE objects
ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in points_of_interest]

# Define the central point and buffer for the region of interest (ROI)
central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)

def get_sentinel_images(start_date, end_date, roi):
    images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)
    return images

def get_cloud_score_images(start_date, end_date, roi):
    images = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)
    return images

def extract_pixel_values(image, points, bands):
    sample = image.select(bands).sampleRegions(
        collection=ee.FeatureCollection(points),
        scale=10,
        geometries=True
    ).getInfo()
    return sample['features']

def convert_timestamp_to_utc_plus_12(timestamp):
    utc_time = datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)
    utc_plus_12_time = utc_time + timedelta(hours=12)
    return utc_plus_12_time.strftime('%Y-%m-%d %H:%M:%S')

def process_data(points, roi, start_date, end_date):
    all_data = []
    bands = ['B2', 'B3', 'B4', 'B8', 'B8A','B11', 'B12']
    cloud_bands = ['cs', 'cs_cdf']
    
    sentinel_images = get_sentinel_images(start_date, end_date, roi)
    cloud_images = get_cloud_score_images(start_date, end_date, roi)
    
    print(f"Processing images from {start_date} to {end_date}")
    sentinel_info = sentinel_images.getInfo()
    
    if not sentinel_info['features']:
        print(f"No Sentinel-2 images found for this date range and region ({start_date} to {end_date}).")
        return pd.DataFrame(all_data)
    else:
        print(f"Found {len(sentinel_info['features'])} Sentinel-2 images.")
    
    for image_info in sentinel_info['features']:
        image = ee.Image(image_info['id'])
        band_values = extract_pixel_values(image, points, bands)
        
        # Extract solar angles from image metadata
        solar_zenith = image_info['properties'].get('MEAN_SOLAR_ZENITH_ANGLE', None)
        solar_azimuth = image_info['properties'].get('MEAN_SOLAR_AZIMUTH_ANGLE', None)
        
        timestamp = image_info['properties']['system:time_start']
        start_time = ee.Date(timestamp)
        end_time = ee.Date(timestamp).advance(1, 'day')
        
        cloud_images_filtered = cloud_images.filterDate(start_time, end_time)
        cloud_info = cloud_images_filtered.getInfo()
        
        if not cloud_info['features']:
            print(f"No Cloud Score images found for the date {datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)} and region.")
            continue
        
        cloud_image = ee.Image(cloud_info['features'][0]['id'])
        cloud_values = extract_pixel_values(cloud_image, points, cloud_bands)
        
        timestamp_str = convert_timestamp_to_utc_plus_12(timestamp)
        
        for i, (band_value, cloud_value) in enumerate(zip(band_values, cloud_values)):
            if band_value['id'] == cloud_value['id']:
                data = {**band_value['properties'], **cloud_value['properties']}
                data['timestamp'] = timestamp_str
                data['point_index'] = i  # Add point index to differentiate points
                data['solar_zenith'] = solar_zenith
                data['solar_azimuth'] = solar_azimuth
                all_data.append(data)
    
    df = pd.DataFrame(all_data)
    
    # Pivot the dataframe
    df_pivoted = df.pivot(index='timestamp', columns='point_index')
    
    # Flatten the multi-level column index
    df_pivoted.columns = ['_'.join(map(str, col)).strip() for col in df_pivoted.columns.values]
    
    # Reset the index to make timestamp a column
    df_pivoted.reset_index(inplace=True)
    
    return df_pivoted

# Process the data
dataframe = process_data(ee_points, roi, '2018-01-01', '2024-06-10')

# Save or display the dataframe
dataframe.to_csv('sentinel_data_with_cloud_and_solar_angles2.csv', index=False)
print(dataframe.head())


Processing images from 2018-01-01 to 2024-06-10
Found 795 Sentinel-2 images.
No Cloud Score images found for the date 2018-08-27 22:28:23.936000+00:00 and region.
No Cloud Score images found for the date 2018-12-13 22:38:15.731000+00:00 and region.
No Cloud Score images found for the date 2019-03-15 22:28:22.062000+00:00 and region.
No Cloud Score images found for the date 2019-06-23 22:28:29.760000+00:00 and region.
No Cloud Score images found for the date 2019-06-26 22:38:26.585000+00:00 and region.
No Cloud Score images found for the date 2019-06-28 22:28:33.999000+00:00 and region.
No Cloud Score images found for the date 2019-07-01 22:38:30.738000+00:00 and region.
No Cloud Score images found for the date 2019-07-03 22:28:30.473000+00:00 and region.
No Cloud Score images found for the date 2019-07-06 22:38:27.176000+00:00 and region.
No Cloud Score images found for the date 2019-07-08 22:28:34.319000+00:00 and region.
No Cloud Score images found for the date 2019-07-11 22:38:30.90

In [ ]:
dataframe.describe()

In [7]:
import ee
from datetime import datetime, timezone

# Initialize the Earth Engine module
ee.Initialize(project='data690-zhouhaomatt')

# Define the central point and buffer for the region of interest (ROI)
central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)

# Define a time range
start_date = '2018-01-01'
end_date = '2024-06-10'

# Get the Sentinel-2 image collection
sentinel_images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi)

# Function to convert timestamp to human-readable date
def convert_timestamp(timestamp):
    return datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)

# Get the list of image IDs and timestamps
image_info = sentinel_images.select('B4').getInfo()['features']
timestamps = [(img['id'], convert_timestamp(img['properties']['system:time_start'])) for img in image_info]

# Print the image IDs and timestamps
for img_id, ts in timestamps:
    print(f"Image ID: {img_id}, Timestamp: {ts}")

# Verify if timestamps are around the same time window
time_windows = [ts.time() for _, ts in timestamps]
print("Time Windows:", time_windows)


Image ID: COPERNICUS/S2_SR_HARMONIZED/20180827T222541_20180827T222807_T59GMM, Timestamp: 2018-08-27 22:28:23.936000+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181213T223659_20181213T223656_T59GMM, Timestamp: 2018-12-13 22:38:15.731000+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181215T222531_20181215T222536_T59GMM, Timestamp: 2018-12-15 22:28:16+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181218T223651_20181218T223734_T59GMM, Timestamp: 2018-12-18 22:38:13+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181220T222539_20181220T222533_T59GMM, Timestamp: 2018-12-20 22:28:20+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181223T223659_20181223T223658_T59GMM, Timestamp: 2018-12-23 22:38:17+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181225T222531_20181225T222541_T59GMM, Timestamp: 2018-12-25 22:28:18+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181228T223701_20181228T223736_T59GMM, Timestamp: 2018-12-28 22:38:15+00:00
Image ID: COPERNICUS/S2_SR_HARMONIZED/20181230T222539_20181230T222